<a href="https://colab.research.google.com/github/vikaspathak0911/MyPython/blob/main/video_segmentation_using_bounding_boxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# --- 2. Install Dependencies ---
%pip install opencv-python ultralytics

import cv2
import os
from collections import deque
from ultralytics import YOLO
import math

print("Libraries installed and imported successfully.")

# --- 3. Configuration ---
# IMPORTANT:
# 1. Replace 'path/to/your/judo_video.mp4' with the actual path to your video
#    in Google Drive.
#    Example: '/content/drive/My Drive/Judo_Videos/my_judo_match.mp4'
# 2. The OUTPUT_DIR will be created in your Google Drive.
#    Example: '/content/drive/My Drive/Judo_Clips_Output'

VIDEO_PATH = '/content/drive/My Drive/Japan vs South Korea .mp4' # <--- CHANGE THIS TO YOUR VIDEO PATH
OUTPUT_DIR = '/content/drive/My Drive/Judo_Throw_Clips_With_Boxes'    # <--- OUTPUT DIRECTORY IN YOUR DRIVE

CLIP_DURATION_SECONDS = 5      # Total clip length (e.g., 4-6 seconds)
PRE_EVENT_SECONDS = 2          # Seconds before the event to include (adjust as needed)
POST_EVENT_SECONDS = CLIP_DURATION_SECONDS - PRE_EVENT_SECONDS # Seconds after the event

# Bounding box ratio threshold for throw detection (height < ratio * width)
# This is the most crucial parameter to tune.
# Experiment with values like 0.8, 0.9, 0.95 depending on your video angles.
BOUNDING_BOX_RATIO_THRESHOLD = 0.9

# New parameters for filtering closest central persons:
# Max distance from frame center (as a fraction of frame diagonal) for a person to be considered "central".
# Tune this value: 0.2 means center of bbox must be within 20% of diagonal from frame center.
MAX_CENTER_DISTANCE_RATIO = 0.25
# Minimum required confidence for person detection (YOLOv8's output confidence)
# If you're not seeing boxes, try lowering this (e.g., to 0.4 or 0.3)
PERSON_CONFIDENCE_THRESHOLD = 0.5

# Bounding box visualization color and thickness for the output clips
# Using Bright Red for clear visibility during debugging. Change to Green (0,255,0) later if desired.
BOUNDING_BOX_COLOR = (0, 0, 255)  # Bright Red color (BGR format: Blue, Green, Red)
BOUNDING_BOX_THICKNESS = 5        # Make it thicker

# --- Ensure output directory exists in Google Drive ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory set to: {OUTPUT_DIR}")

# --- 4. Initialize YOLOv8 Model ---
yolo_model = YOLO('yolov8n.pt') # You can try 'yolov8s.pt' if you have a good GPU runtime.

person_class_id = None
for class_id, class_name in yolo_model.names.items():
    if class_name == 'person':
        person_class_id = class_id
        break

if person_class_id is None:
    raise ValueError("Could not find 'person' class in the YOLO model's names. Check model configuration.")
print(f"YOLOv8 model loaded. 'person' class ID: {person_class_id}")


# --- 5. Helper Functions ---

def get_all_person_bounding_boxes(frame, model, person_id, conf_thresh):
    """
    Uses YOLOv8 to detect ALL people in a frame and return their bounding boxes
    with confidence scores.
    Returns a list of dictionaries: [{'bbox': [x1,y1,x2,y2], 'conf': conf_score}]
    """
    results = model(frame, verbose=False, conf=conf_thresh, device=0)
    person_detections = []
    for r in results:
        for *xyxy, conf, cls in r.boxes.data:
            if int(cls) == person_id:
                person_detections.append({'bbox': [int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])],
                                          'conf': float(conf)})
    return person_detections

def calculate_bbox_dimensions_and_center(bbox):
    """Calculates width, height, area, and center from [x_min, y_min, x_max, y_max]."""
    x_min, y_min, x_max, y_max = bbox
    width = x_max - x_min
    height = y_max - y_min
    area = width * height
    center_x = (x_min + x_max) // 2
    center_y = (y_min + y_max) // 2
    return width, height, area, (center_x, center_y)

def select_closest_central_persons(person_detections, frame_center, frame_diagonal_length, max_center_dist_ratio):
    """
    Filters and selects up to two closest persons who are also near the center of the frame.
    Returns a list of dictionaries, each containing 'bbox', 'width', 'height', 'area', 'center'.
    """
    central_persons_data = []

    for detection in person_detections:
        bbox = detection['bbox']
        width, height, area, bbox_center = calculate_bbox_dimensions_and_center(bbox)

        if width <= 0 or height <= 0 or area <= 0:
            continue

        dist_from_frame_center = math.sqrt(
            (bbox_center[0] - frame_center[0])**2 + (bbox_center[1] - frame_center[1])**2
        )

        if dist_from_frame_center <= max_center_dist_ratio * frame_diagonal_length:
            central_persons_data.append({
                'bbox': bbox,
                'area': area,
                'dist_from_center': dist_from_frame_center,
                'width': width,
                'height': height,
                'center': bbox_center
            })

    central_persons_data.sort(key=lambda x: x['area'], reverse=True)

    return central_persons_data[:2]

def extract_clip_with_frames(output_dir, frames, fps, clip_index, video_dimensions):
    """Extracts and saves a video clip using a list of pre-processed frames."""
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4, commonly used
    clip_filename = os.path.join(output_dir, f"throw_clip_{clip_index:04d}.mp4")

    out = cv2.VideoWriter(clip_filename, fourcc, fps, video_dimensions)

    frames_written = 0
    for frame in frames:
        if frame is None:
            print(f"Warning: Attempted to write a None frame for clip {clip_index}")
            continue
        try:
            out.write(frame)
            frames_written += 1
        except Exception as e:
            print(f"Error writing frame to {clip_filename}: {e}")
            break

    out.release()
    print(f"Saved clip: {clip_filename} (Wrote {frames_written} frames)")


# --- 6. Main Processing Logic ---

def process_video_for_throws(video_path, output_dir, clip_duration, pre_event_duration, post_event_duration,
                              bbox_ratio_threshold, yolo_model, person_class_id,
                              max_center_dist_ratio, person_conf_threshold,
                              bounding_box_color, bounding_box_thickness):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}. Please check the path and permissions.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_dimensions = (video_width, video_height)

    frame_center = (video_width // 2, video_height // 2)
    frame_diagonal_length = math.sqrt(video_width**2 + video_height**2)

    print(f"Video opened: {video_path}")
    print(f"FPS: {fps}, Total Frames: {total_frames}, Dimensions: {video_dimensions}")
    print(f"Frame Center: {frame_center}, Diagonal Length: {frame_diagonal_length:.2f}")


    pre_event_frames = int(pre_event_duration * fps)
    post_event_frames = int(post_event_duration * fps)
    total_clip_frames = int(clip_duration * fps)

    clip_count = 0
    event_detected_frame = -1

    current_frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video stream or read error.")
            break

        if event_detected_frame != -1 and current_frame_num < event_detected_frame + post_event_frames:
             current_frame_num += 1
             continue

        all_person_detections = get_all_person_bounding_boxes(frame, yolo_model, person_class_id, person_conf_threshold)
        selected_persons_data = select_closest_central_persons(
            all_person_detections, frame_center, frame_diagonal_length, MAX_CENTER_DISTANCE_RATIO
        )

        throw_detected_in_frame = False
        relevant_bboxes = []
        for person_data in selected_persons_data:
            bbox = person_data['bbox']
            relevant_bboxes.append(bbox)
            width = person_data['width']
            height = person_data['height']
            if width > 0 and height > 0:
                if height < bbox_ratio_threshold * width:
                    throw_detected_in_frame = True
                    break

        if throw_detected_in_frame:
            print(f"\nPotential throw detected at frame: {current_frame_num}")
            event_detected_frame = current_frame_num
            clip_count += 1

            clip_start_frame = max(0, event_detected_frame - pre_event_frames)
            clip_end_frame = clip_start_frame + total_clip_frames - 1
            clip_end_frame = min(clip_end_frame, total_frames - 1)

            actual_clip_length_frames = clip_end_frame - clip_start_frame + 1
            if actual_clip_length_frames < total_clip_frames:
                 clip_start_frame = max(0, clip_end_frame - total_clip_frames + 1)
                 actual_clip_length_frames = clip_end_frame - clip_start_frame + 1

            print(f"  Attempting to extract clip {clip_count}: from frame {clip_start_frame} to {clip_end_frame} (Actual length: {actual_clip_length_frames} frames)")

            frames_for_clip = []
            temp_cap = cv2.VideoCapture(video_path)
            temp_cap.set(cv2.CAP_PROP_POS_FRAMES, clip_start_frame)

            for i in range(clip_start_frame, clip_end_frame + 1):
                ret_temp, frame_temp = temp_cap.read()
                if not ret_temp:
                    print(f"Warning: Could not read frame {i} for clip {clip_count}. Ending clip early.")
                    break

                # Re-run detection for each frame in the clip to ensure boxes are consistent
                all_person_detections_clip = get_all_person_bounding_boxes(frame_temp, yolo_model, person_class_id, person_conf_threshold)
                selected_persons_data_clip = select_closest_central_persons(
                    all_person_detections_clip, frame_center, frame_diagonal_length, MAX_CENTER_DISTANCE_RATIO
                )

                # --- Draw bounding boxes on the frame_temp ---
                if selected_persons_data_clip:
                    # Print statement to confirm drawing for *this specific frame*
                    print(f"    Drawing {len(selected_persons_data_clip)} boxes on frame {i} of clip {clip_count}")
                    for person_data_clip in selected_persons_data_clip:
                        bbox_clip = person_data_clip['bbox']
                        x_min_clip, y_min_clip, x_max_clip, y_max_clip = bbox_clip
                        # Robustly clamp coordinates to be within frame bounds
                        x_min_clip = max(0, x_min_clip)
                        y_min_clip = max(0, y_min_clip)
                        x_max_clip = min(video_width - 1, x_max_clip) # -1 because max coordinate is width-1
                        y_max_clip = min(video_height - 1, y_max_clip) # -1 because max coordinate is height-1

                        cv2.rectangle(frame_temp, (x_min_clip, y_min_clip), (x_max_clip, y_max_clip),
                                      bounding_box_color, bounding_box_thickness)
                else:
                    # Print statement if no persons are selected for drawing on this frame
                    print(f"    No persons selected for drawing on frame {i} of clip {clip_count}")

                frames_for_clip.append(frame_temp)
            temp_cap.release()

            extract_clip_with_frames(output_dir, frames_for_clip, fps, clip_count, video_dimensions)

        current_frame_num += 1

    cap.release()
    print("\nVideo processing finished. All potential clips with bounding boxes extracted.")

# --- 7. Run the processing ---
if __name__ == "__main__":
    print("Starting judo throw clip extraction with bounding box visualization...")
    process_video_for_throws(
        video_path=VIDEO_PATH,
        output_dir=OUTPUT_DIR,
        clip_duration=CLIP_DURATION_SECONDS,
        pre_event_duration=PRE_EVENT_SECONDS,
        post_event_duration=POST_EVENT_SECONDS,
        bbox_ratio_threshold=BOUNDING_BOX_RATIO_THRESHOLD,
        yolo_model=yolo_model,
        person_class_id=person_class_id,
        max_center_dist_ratio=MAX_CENTER_DISTANCE_RATIO,
        person_conf_threshold=PERSON_CONFIDENCE_THRESHOLD,
        bounding_box_color=BOUNDING_BOX_COLOR,
        bounding_box_thickness=BOUNDING_BOX_THICKNESS
    )
    print("Process complete. Check your Google Drive for the clips with bounding boxes.")

Streaming output truncated to the last 5000 lines.
    Drawing 2 boxes on frame 24315 of clip 184
    Drawing 2 boxes on frame 24316 of clip 184
    Drawing 2 boxes on frame 24317 of clip 184
    Drawing 2 boxes on frame 24318 of clip 184
    Drawing 2 boxes on frame 24319 of clip 184
    Drawing 2 boxes on frame 24320 of clip 184
    Drawing 2 boxes on frame 24321 of clip 184
    Drawing 2 boxes on frame 24322 of clip 184
    Drawing 2 boxes on frame 24323 of clip 184
    Drawing 2 boxes on frame 24324 of clip 184
    Drawing 2 boxes on frame 24325 of clip 184
    Drawing 2 boxes on frame 24326 of clip 184
    Drawing 2 boxes on frame 24327 of clip 184
    Drawing 2 boxes on frame 24328 of clip 184
    Drawing 2 boxes on frame 24329 of clip 184
    Drawing 2 boxes on frame 24330 of clip 184
    Drawing 2 boxes on frame 24331 of clip 184
    Drawing 2 boxes on frame 24332 of clip 184
    Drawing 2 boxes on frame 24333 of clip 184
    Drawing 2 boxes on frame 24334 of clip 184
    Drawi

KeyboardInterrupt: 

In [ ]:
# --- 2. Install Dependencies ---
%pip install opencv-python ultralytics

import cv2
import os
from collections import deque
from ultralytics import YOLO
import math

print("Libraries installed and imported successfully.")



Libraries installed and imported successfully.


In [ ]:
# --- 3. Configuration ---
VIDEO_PATH = '/content/drive/My Drive/Japan vs South Korea .mp4' # <--- CHANGE THIS TO YOUR VIDEO PATH
OUTPUT_DIR = '/content/drive/My Drive/Judo_Throw_Clips'    # <--- OUTPUT DIRECTORY IN YOUR DRIVE

CLIP_DURATION_SECONDS = 10    # Total clip length (e.g., 4-6 seconds)
PRE_EVENT_SECONDS = 5       # Seconds before the event to include
POST_EVENT_SECONDS = CLIP_DURATION_SECONDS - PRE_EVENT_SECONDS

BOUNDING_BOX_RATIO_THRESHOLD = 0.9 # Tune this: height < ratio * width for throw detection

# New parameters for filtering closest central persons:
# Max distance from frame center (as a fraction of frame diagonal) for a person to be considered "central".
# Tune this value: 0.2 means center of bbox must be within 20% of diagonal from frame center.
MAX_CENTER_DISTANCE_RATIO = 0.25
# Minimum required confidence for person detection
PERSON_CONFIDENCE_THRESHOLD = 0.5

# Bounding box visualization color and thickness for the output clips
BOUNDING_BOX_COLOR = (0, 255, 0)  # Green color (BGR format: Blue, Green, Red)
BOUNDING_BOX_THICKNESS = 2

# --- Ensure output directory exists in Google Drive ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory set to: {OUTPUT_DIR}")



Output directory set to: /content/drive/My Drive/Judo_Throw_Clips


In [ ]:
# --- 4. Initialize YOLOv8 Model ---
yolo_model = YOLO('yolov8n.pt') # Using nano for speed. Consider 'yolov8s.pt' for better accuracy.

person_class_id = None
for class_id, class_name in yolo_model.names.items():
    if class_name == 'person':
        person_class_id = class_id
        break

if person_class_id is None:
    raise ValueError("Could not find 'person' class in the YOLO model's names. Check model configuration.")
print(f"YOLOv8 model loaded. 'person' class ID: {person_class_id}")

YOLOv8 model loaded. 'person' class ID: 0


In [ ]:
# --- 5. Helper Functions ---

def get_all_person_bounding_boxes(frame, model, person_id, conf_thresh):
    """
    Uses YOLOv8 to detect ALL people in a frame and return their bounding boxes
    with confidence scores.
    Returns a list of dictionaries: [{'bbox': [x1,y1,x2,y2], 'conf': conf_score}]
    """
    results = model(frame, verbose=False, conf=conf_thresh, device=0) # conf threshold applied here
    person_detections = []
    for r in results:
        for *xyxy, conf, cls in r.boxes.data:
            if int(cls) == person_id:
                person_detections.append({'bbox': [int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])],
                                          'conf': float(conf)})
    return person_detections

def calculate_bbox_dimensions_and_center(bbox):
    """Calculates width, height, area, and center from [x_min, y_min, x_max, y_max]."""
    x_min, y_min, x_max, y_max = bbox
    width = x_max - x_min
    height = y_max - y_min
    area = width * height
    center_x = (x_min + x_max) // 2
    center_y = (y_min + y_max) // 2
    return width, height, area, (center_x, center_y)

def distance(p1, p2):
    """Calculates Euclidean distance between two points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def select_closest_central_persons(person_detections, frame_center, frame_diagonal_length, max_center_dist_ratio):
    """
    Filters and selects up to two closest persons who are also near the center of the frame.
    """
    central_persons_data = [] # Stores (bbox, area, distance_from_center) for central persons

    for detection in person_detections:
        bbox = detection['bbox']
        width, height, area, bbox_center = calculate_bbox_dimensions_and_center(bbox)

        # Skip invalid bounding boxes
        if width <= 0 or height <= 0 or area <= 0:
            continue

        dist_from_frame_center = distance(bbox_center, frame_center)

        # Check if the person is "central" enough
        if dist_from_frame_center <= max_center_dist_ratio * frame_diagonal_length:
            central_persons_data.append({
                'bbox': bbox,
                'area': area,
                'dist_from_center': dist_from_frame_center,
                'width': width,
                'height': height
            })

    # Sort central persons by area in descending order (largest area means likely closest)
    central_persons_data.sort(key=lambda x: x['area'], reverse=True)

    # Return the top two (or fewer if less are available) bounding boxes
    return [p['bbox'] for p in central_persons_data[:2]]


def extract_clip(video_path, output_dir, start_frame, end_frame, fps, clip_index, video_dimensions):
    """Extracts and saves a video clip."""
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    clip_filename = os.path.join(output_dir, f"throw_clip_{clip_index:04d}.mp4")

    out = cv2.VideoWriter(clip_filename, fourcc, fps, video_dimensions)

    current_frame_num = start_frame
    frames_written = 0
    while cap.isOpened() and current_frame_num <= end_frame:
        ret, frame = cap.read()
        if not ret:
            # This can happen if the requested end_frame is beyond the actual video length
            # or if there's a read error.
            # print(f"Warning: Reached end of video or failed to read at frame {current_frame_num}")
            break
        try:
            out.write(frame)
            frames_written += 1
        except Exception as e:
            print(f"Error writing frame {current_frame_num} to {clip_filename}: {e}")
            break

        current_frame_num += 1

    out.release()
    cap.release()
    print(f"Saved clip: {clip_filename} (Frames: {start_frame}-{current_frame_num-1}, Written: {frames_written})")





In [ ]:
# --- 6. Main Processing Logic ---

def process_video_for_throws(video_path, output_dir, clip_duration, pre_event_duration, post_event_duration,
                              bbox_ratio_threshold, yolo_model, person_class_id,
                              max_center_dist_ratio, person_conf_threshold):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}. Please check the path and permissions.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_dimensions = (video_width, video_height)

    # Calculate frame center and diagonal length for centrality checks
    frame_center = (video_width // 2, video_height // 2)
    frame_diagonal_length = math.sqrt(video_width**2 + video_height**2)

    print(f"Video opened: {video_path}")
    print(f"FPS: {fps}, Total Frames: {total_frames}, Dimensions: {video_dimensions}")
    print(f"Frame Center: {frame_center}, Diagonal Length: {frame_diagonal_length:.2f}")


    pre_event_frames = int(pre_event_duration * fps)
    post_event_frames = int(post_event_duration * fps)
    total_clip_frames = int(clip_duration * fps)

    clip_count = 0
    event_detected_frame = -1

    current_frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video stream or read error.")
            break

        # If we are within a recently extracted clip's timeframe, skip processing
        if event_detected_frame != -1 and current_frame_num < event_detected_frame + post_event_frames:
             current_frame_num += 1
             continue

        # 1. Get ALL person detections first
        all_person_detections = get_all_person_bounding_boxes(frame, yolo_model, person_class_id, person_conf_threshold)

        # 2. Filter and select the two closest central persons
        selected_bboxes = select_closest_central_persons(
            all_person_detections, frame_center, frame_diagonal_length, MAX_CENTER_DISTANCE_RATIO
        )

        throw_detected_in_frame = False
        if len(selected_bboxes) >= 1: # We need at least one person to check the throw condition
            for bbox in selected_bboxes:
                width, height = calculate_bbox_dimensions_and_center(bbox)[0:2] # Get width and height

                if width > 0 and height > 0:
                    if height < bbox_ratio_threshold * width:
                        throw_detected_in_frame = True
                        # print(f"  Triggering bbox (W={width}, H={height}, Ratio={height/width:.2f})")
                        break # Found one person satisfying the condition, so consider it a potential event

        if throw_detected_in_frame:
            print(f"\nPotential throw detected at frame: {current_frame_num}")
            event_detected_frame = current_frame_num
            clip_count += 1

            clip_start_frame = max(0, event_detected_frame - pre_event_frames)
            clip_end_frame = clip_start_frame + total_clip_frames - 1
            clip_end_frame = min(clip_end_frame, total_frames - 1)

            actual_clip_length_frames = clip_end_frame - clip_start_frame + 1
            if actual_clip_length_frames < total_clip_frames:
                 clip_start_frame = max(0, clip_end_frame - total_clip_frames + 1)
                 actual_clip_length_frames = clip_end_frame - clip_start_frame + 1 # Recalculate

            print(f"  Attempting to extract clip {clip_count}: from frame {clip_start_frame} to {clip_end_frame} (Actual length: {actual_clip_length_frames} frames)")

            extract_clip(video_path, output_dir, clip_start_frame, clip_end_frame, fps, clip_count, video_dimensions)

        current_frame_num += 1

    cap.release()
    print("\nVideo processing finished. All potential clips extracted.")


In [ ]:
# --- 7. Run the processing ---
if __name__ == "__main__":
    print("Starting judo throw clip extraction with refined person detection...")
    process_video_for_throws(
        video_path=VIDEO_PATH,
        output_dir=OUTPUT_DIR,
        clip_duration=CLIP_DURATION_SECONDS,
        pre_event_duration=PRE_EVENT_SECONDS,
        post_event_duration=POST_EVENT_SECONDS,
        bbox_ratio_threshold=BOUNDING_BOX_RATIO_THRESHOLD,
        yolo_model=yolo_model,
        person_class_id=person_class_id,
        max_center_dist_ratio=MAX_CENTER_DISTANCE_RATIO,
        person_conf_threshold=PERSON_CONFIDENCE_THRESHOLD
    )
    print("Process complete. Check your Google Drive for the clips.")

Starting judo throw clip extraction with refined person detection...
Video opened: /content/drive/My Drive/Japan vs South Korea .mp4
FPS: 25.0, Total Frames: 59520, Dimensions: (1280, 720)
Frame Center: (640, 360), Diagonal Length: 1468.60

Potential throw detected at frame: 513
  Attempting to extract clip 1: from frame 388 to 637 (Actual length: 250 frames)
Saved clip: /content/drive/My Drive/Judo_Throw_Clips/throw_clip_0001.mp4 (Frames: 388-637, Written: 250)

Potential throw detected at frame: 638
  Attempting to extract clip 2: from frame 513 to 762 (Actual length: 250 frames)
Saved clip: /content/drive/My Drive/Judo_Throw_Clips/throw_clip_0002.mp4 (Frames: 513-762, Written: 250)

Potential throw detected at frame: 1350
  Attempting to extract clip 3: from frame 1225 to 1474 (Actual length: 250 frames)
Saved clip: /content/drive/My Drive/Judo_Throw_Clips/throw_clip_0003.mp4 (Frames: 1225-1474, Written: 250)

Potential throw detected at frame: 1475
  Attempting to extract clip 4: f

KeyboardInterrupt: 